In [117]:
import pandas as pd
from datasets import load_dataset, Dataset
import os
import re
import torch
from sklearn.model_selection import train_test_split
import requests
import random

### DATASET

In [118]:
#si importano i dataset
train_df = pd.read_csv("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/data_set/train_set.csv")
val_df = pd.read_csv("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/data_set/val_set.csv")

In [ ]:
print(f'Training set di dimensioni: {len(train_df)} - Variabili: {train_df.columns.to_list()}')
print(f'Validation set di dimensioni: {len(val_df)} - Variabili: {val_df.columns.to_list()}')

In [120]:
#ordinamento solo in caso di addestramento su curriculum o anti-curriculum
sorted_df = train_df.sort_values(by='gulpease', ascending=False)

In [121]:
#Trasformiamo in formato Dataset i dati precedentemente importati
tr_set = Dataset.from_pandas(sorted_df[:5000])
val_set = Dataset.from_pandas(val_df[:250])

In [122]:
tr_set = tr_set.shuffle(seed=5) #shuffle se ordinamento random

In [ ]:
tr_set[0:5]

### TOKENIZATION

In questa sezione si importa il tokenizzatore col quale si tokenizza ciascuna frase nel formato necessario per Bert, alla fine si otterrà un dataset nel formato corretto con tutte le features necessarie per il training

In [124]:
import tokenizers
import transformers
from transformers import BertTokenizer


In [125]:
#si importa il tokenizzatore già configurato (in questo caso: bert-base-italian-cased)
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")


In [ ]:
#facciamo l'encoding di tutto il dataset tokenizzando frase per frase
def encode(sample):
    return tokenizer(sample["text"], padding=True, truncation=True, max_length=128, return_special_tokens_mask=True) 


train_set = tr_set.map(encode, batched=True)
test_set = val_set.map(encode, batched=True)
train_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])
test_set.set_format('torch', columns=["input_ids", "attention_mask", "token_type_ids"])


In [ ]:
train_set

In [ ]:
test_set

### TRAINING DI BERT

Si procede al training di Bert. Il modello dovrà partire da uno stato iniziale con pesi random, per questo non si importa il modello già addestrato, ma si configura semplicemente l'architettura la sua architettura per poi addestrarlo da zero. Si definisce poi una strategia di training e i suoi argomenti per poi addestrare il modello sul trask di Language Modeling. 

In [129]:
from transformers import Trainer, TrainingArguments, TrainerCallback, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling, set_seed


In [ ]:
#si importano configurazione e modello
model_name = "prajjwal1/bert-medium"
model_config = BertConfig.from_pretrained(model_name)
model = BertForMaskedLM(model_config)
print(model_config)

In [ ]:
#si importa un modello già inizializzato
model_path = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale"
model.load_state_dict(torch.load(f'{model_path}/initial_model.bin'))
model.resize_token_embeddings(len(tokenizer))

In [ ]:
model.config

In [133]:
#usiamo il datacollator per fare le batch per il training
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.2, return_tensors="pt")

In [ ]:
datacollator

In [142]:
#definisco alcune funzione di callback
from transformers import TrainerCallback
from transformers.trainer_callback import TrainerControl, TrainerState
from transformers.training_args import TrainingArguments
from transformers.integrations import TensorBoardCallback

 



#callback per incrementare il numero di step tra ogni salvataggio e salvataggio customizzato
class IncrementSaveSteps(TrainerCallback):
    def __init__(self, increase_steps, save_steps, checkpoint_path):
        self.increase_steps = increase_steps
        self.save_steps = save_steps
        self.checkpoint_path = checkpoint_path
  
    def on_step_end(self, args, state, control, logs=None, **kwargs):
        current_step = state.global_step
        if current_step % self.save_steps == 0:
            self.save_checkpoint(state, **kwargs)
        if current_step in self.increase_steps:
            self.save_steps = current_step
            state.logging_steps = self.save_steps
            print(f'Changed checkpoint and logging steps to {self.save_steps}')

    def save_checkpoint(self, state, **kwargs):
        current_step = state.global_step
        path_name = f'{self.checkpoint_path}/checkpoint-step{current_step}'
        if not os.path.exists(path_name):
            os.makedirs(path_name)
        kwargs['model'].save_pretrained(path_name)
        state.save_to_json(os.path.join(path_name, "trainer_state.json"))
        kwargs['model'].config.save_pretrained(path_name)
        if hasattr(kwargs['model'], 'generation_config'):
            kwargs['model'].generation_config.save_pretrained(path_name)
        print(f'Checkpoint has been saved for step number {current_step}. Current step_size = {self.save_steps}')


#callback per salvare le metriche a ogni evaluation
class HistoryLogger(TrainerCallback):
    def __init__(self, dir_path):
        self.dir_path = os.path.join(dir_path, "history_log.json")

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        with open(self.dir_path, 'a') as f:
                f.write(json.dumps(metrics) + '\n')
                print(f"History log file has been updated with step's {state.global_step} metrics.")

#callback per tensorboard
tensorboard_callback = TensorBoardCallback()

            

In [143]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


#da modificare per masked language modeling
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    mask = labels != -100
    labels = labels[mask]
    predictions = predictions[mask]
    accuracy = accuracy_score(labels, predictions)
    cross_entropy = torch.nn.CrossEntropyLoss()(torch.tensor(logits).permute(0, 2, 1), torch.tensor(labels))
    perplexity = torch.exp(cross_entropy)
    return {
        'accuracy': accuracy,
        'perplexity': perplexity.item()
    }


In [144]:
results_path = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/training/prova"
output_dir = os.path.join(results_path, "my_pretrained_model")

In [145]:
#argomenti per il training
training_args = TrainingArguments(
    output_dir = output_dir,
    eval_strategy="steps", 
    overwrite_output_dir=True,
    per_device_train_batch_size=16, #64, 
    per_device_eval_batch_size=16, #64, 
    num_train_epochs=1,
    logging_steps=2,
    save_strategy="no",
    eval_steps=2000,
    load_best_model_at_end=False,
    save_steps=2,
    seed=42, 
    )

In [146]:
set_seed(training_args.seed)

In [ ]:
checkpoint_path = os.path.join(results_path, "checkpoints")
steps_increments = [4, 16, 256, 2048]

#si definiscono i parametri del Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=datacollator,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics,
    callbacks=[IncrementSaveSteps(steps_increments, 2, checkpoint_path), HistoryLogger(dir_path=results_path), tensorboard_callback], 
)


In [ ]:
trainer.train() #si addestra